[Learning Lukasiewicz Logic Fragments by Quadratic Programming](http://ecmlpkdd2017.ijs.si/papers/paperID223.pdf)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cvxpy as cp

# 1. Toy problem

---
メモ

* 各 p に対してデータの数が揃っている等の理由により，実装がシンプルになっているので，他の問題に直ちに適用することはできない

## 1.1 データの準備

In [2]:
L_1 = [(.1, .5, -1), (.4, .4, -1), (.3, .8, 1), (.9, .7, 1)]
L_2 = [(.1, .3, -1), (.6, .4, -1), (.2, .8, 1), (.7, .6, 1)]
L_3 = [(.4, .2, -1), (.9, .3, -1), (.2, .6, 1), (.5, .7, 1)]

U = [(.1, .5), (.3, .7), (.5, .4), (.8, .3), (.9, .2), (1, .5)] # U_1 = U_2 = U_3 = U ということでいいのか


display(L_1)
display(L_2)
display(L_3)
print()
display(U)

[(0.1, 0.5, -1), (0.4, 0.4, -1), (0.3, 0.8, 1), (0.9, 0.7, 1)]

[(0.1, 0.3, -1), (0.6, 0.4, -1), (0.2, 0.8, 1), (0.7, 0.6, 1)]

[(0.4, 0.2, -1), (0.9, 0.3, -1), (0.2, 0.6, 1), (0.5, 0.7, 1)]

[(0.1, 0.5), (0.3, 0.7), (0.5, 0.4), (0.8, 0.3), (0.9, 0.2), (1, 0.5)]

In [3]:
U = np.array(U).T

display(U)

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

In [4]:
x_L_1, y_L_1 = np.array([[x_l[0], x_l[1]] for x_l in L_1]).T, np.array([[y_l[2]] for y_l in L_1]).T
x_L_2, y_L_2 = np.array([[x_l[0], x_l[1]] for x_l in L_2]).T, np.array([[y_l[2]] for y_l in L_2]).T
x_L_3, y_L_3 = np.array([[x_l[0], x_l[1]] for x_l in L_3]).T, np.array([[y_l[2]] for y_l in L_3]).T

display(x_L_1)
display(y_L_1)
print()
display(x_L_2)
display(y_L_2)
print()
display(x_L_3)
display(y_L_3)

array([[0.1, 0.4, 0.3, 0.9],
       [0.5, 0.4, 0.8, 0.7]])

array([[-1, -1,  1,  1]])

array([[0.1, 0.6, 0.2, 0.7],
       [0.3, 0.4, 0.8, 0.6]])

array([[-1, -1,  1,  1]])

array([[0.4, 0.9, 0.2, 0.5],
       [0.2, 0.3, 0.6, 0.7]])

array([[-1, -1,  1,  1]])

In [5]:
x_L = [x_L_1, x_L_2, x_L_3]
x_L

[array([[0.1, 0.4, 0.3, 0.9],
        [0.5, 0.4, 0.8, 0.7]]),
 array([[0.1, 0.6, 0.2, 0.7],
        [0.3, 0.4, 0.8, 0.6]]),
 array([[0.4, 0.9, 0.2, 0.5],
        [0.2, 0.3, 0.6, 0.7]])]

In [6]:
y_L = [y_L_1, y_L_2, y_L_3]
y_L

[array([[-1, -1,  1,  1]]),
 array([[-1, -1,  1,  1]]),
 array([[-1, -1,  1,  1]])]

In [7]:
x_U_1 = U
x_U_2 = U
x_U_3 = U

x_U = [x_U_1, x_U_2, x_U_3]

display(x_U)

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])]

In [8]:
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

In [9]:
x_S_1 = np.hstack([x_U_1, x_L_1])
x_S_2 = np.hstack([x_U_2, x_L_2])
x_S_3 = np.hstack([x_U_3, x_L_3])

x_S = [x_S_1, x_S_2, x_S_3]

x_S

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.4, 0.3, 0.9],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.4, 0.8, 0.7]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.6, 0.2, 0.7],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.3, 0.4, 0.8, 0.6]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.4, 0.9, 0.2, 0.5],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.2, 0.3, 0.6, 0.7]])]

In [10]:
# 6 x 12 の行列
M_1 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M_2 = np.array([[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1]])

M = [M_1, M_2]
print(M)

[array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]]), array([[-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  1]])]


In [11]:
# 6 x 1 の行列（列ベクトル）
q_1 = np.array([[1, 1, 1, 1, 1, 1]]).T
q_2 = np.array([[1, 1, 1, 1, 1, 1]]).T

q = [q_1, q_2]
print(q)

[array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]]), array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]])]


## 1.2 目的関数の実装

メモ

* cp.quad_form で計算できるように，シグマ記号を二次形式で書き換える

In [12]:
# 通常の内積

def kernel_function(x1, x2):
    return x1.T @ x2

k = kernel_function


# 変数 
lambda_j_l = cp.Variable(shape=(3, 4), nonneg=True) # j \in {1, 2, 3}, l \in {1, 2, 3, 4}
lambda_h_i = cp.Variable(shape=(2, 6), nonneg=True) # h \in {1, 2}, i \in {1, 2, 3, 4, 5, 6}
eta_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
eta_hat_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

lambda_j_l, lambda_h_i, eta_j_s, eta_hat_j_s

(Variable((3, 4), nonneg=True),
 Variable((2, 6), nonneg=True),
 Variable((3, 10), nonneg=True),
 Variable((3, 10), nonneg=True))

In [13]:
len_j = 3
len_l = 4 

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

### 1.2.1 目的関数

In [14]:
obj_func = 0


for j in range(len_j):
    A = np.zeros((len_l, len_l))
    for l in range(len_l):
        for l_prime in range(len_l):
            A[l, l_prime] = y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])
    # print(f'A \n{A}\n')

    obj_func += 4 * cp.quad_form(lambda_j_l[0, :], A)


    tmp_obj = 0
    for h in range(len_h):
        for h_prime in range(len_h):
            # 各 h について，I_h （h が取る要素の数）がそれぞれ違えば，ここは変更しないといけない
            B = np.zeros((len_I_h, len_I_h))
            for i in range(len_I_h):
                for i_prime in range(len_I_h):
                    for u in range(len_u):
                        for u_prime in range(len_u):
                            B[i, i_prime] = M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])
            tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), B])
            tmp2 = np.hstack([B.T, np.zeros((len_I_h, len_I_h))])
            B = np.vstack([tmp1, tmp2])
            tmp_vars = cp.hstack([lambda_h_i[h], lambda_h_i[h_prime]])
            cp.quad_form(tmp_vars, B)
            # tmp_obj += cp.quad_form(tmp_vars, B)
            tmp_obj += cp.quad_form(tmp_vars, B, assume_PSD=True)
    
    obj_func += tmp_obj


    D = np.zeros((len_s, len_s))
    for s in range(len_s):
        for s_prime in range(len_s):
            D[s, s_prime] = k(x_S[j][:, s], x_S[j][:, s_prime])
    # print(f'D \n{D}\n')
    
    obj_func += cp.quad_form(eta_j_s[j, :] - eta_hat_j_s[j, :], D)


    tmp_obj = 0
    for h in range(len_h):
        E = np.zeros((len_l, len_I_h))
        for l in range(len_l):
            for i in range(len_I_h):
                tmp = 0
                for u in range(len_u):
                    tmp += y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])
                E[l, i] = tmp
        tmp1 = np.hstack([np.zeros((len_l, len_l)), E])
        tmp2 = np.hstack([E.T, np.zeros((len_I_h, len_I_h))])
        E = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_j_l[j], lambda_h_i[h]])
        # cp.quad_form(tmp_vars, E)
        tmp_obj += cp.quad_form(tmp_vars, E, assume_PSD=True)
    
    # ここで -4 でかけると 全体で concavity がなくなる
    # obj_func += (-4) * tmp_obj
    obj_func += 4 * tmp_obj


    F = np.zeros((len_l, len_s))
    for l in range(len_l):
        for s in range(len_s):
            F[l, s] = y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])
    tmp1 = np.hstack([np.zeros((len_l, len_l)), F])
    tmp2 = np.hstack([F.T, np.zeros((len_s, len_s))])
    F = np.vstack([tmp1, tmp2])
    tmp_vars = cp.hstack([lambda_j_l[j], eta_j_s[j] - eta_hat_j_s[j]])

    # obj_func += 4 * cp.quad_form(tmp_vars, F)
    obj_func += 4 * cp.quad_form(tmp_vars, F, assume_PSD=True)
    

    tmp_obj = 0
    for h in range(len_h):
        G = np.zeros((len_I_h, len_s))
        for i in range(len_I_h):
            for s in range(len_s):
                tmp = 0
                for u in range(len_u):
                    tmp += M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
                G[i, s] = tmp
        tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), G])
        tmp2 = np.hstack([G.T, np.zeros((len_s, len_s))])
        G = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_h_i[h], eta_j_s[j] - eta_hat_j_s[j]])
        # cp.quad_form(tmp_vars, G)
        tmp_obj += cp.quad_form(tmp_vars, G, assume_PSD=True)

    # ここで，-2 でかけると，全体で concavity がなくなる
    # obj_func += (-2) * tmp_obj
    obj_func += 2 * tmp_obj


for j in range(len_j):
    for l in range(len_l):
        obj_func += lambda_j_l[j, l]

for h in range(len_h):
    for i in range(len_I_h):
        tmp = 0
        for u in range(len_u):
            tmp += M[h][i, u]
        
        obj_func += lambda_h_i[h, i] * (1/2 * tmp + q[h][i,0])

tmp = 0
for j in range(len_j):
    for s in range(len_s):
        tmp += eta_j_s[j, s] + eta_hat_j_s[j, s]
obj_func += -1/2 * tmp


obj_func = (-1/2) * obj_func

display(obj_func)

Expression(CONCAVE, UNKNOWN, (1, 1))

In [16]:
print(obj_func)

-0.5 @ (4.0 @ QuadForm(var1[0, 0:4], [[ 0.26  0.24 -0.43 -0.44]
 [ 0.24  0.32 -0.44 -0.64]
 [-0.43 -0.44  0.73  0.83]
 [-0.44 -0.64  0.83  1.3 ]]) + QuadForm(Hstack(var2[0, 0:6], var2[0, 0:6]), psd_wrap([[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.25]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.25 0.   0.   0.   0.   0.   0.  ]])) + QuadForm(Hstack(var2[0, 0:6], var2[1

#### 

$\sum_{s, s'} (\eta_{j_{s}} - \bar{\eta}_{j_{s}})(\eta_{j_{s'}} - \bar{\eta}_{j_{s'}}) k_j(\bold{x}_s, \bold{x}_{s'})$

####

$\sum_{l, l'} \lambda_{j_l} \lambda_{j_{l'}} y_{j_l} y_{j_{l'}} k_j(\bold x_l, \bold x_{l'})$

#### 

$\sum_{l, s} \lambda_{j_l} y_l (\eta_{j_{s}} - \bar{\eta}_{j_{s}}) k_j(\bold{x}_l, \bold{x}_{s})$

In [17]:
len_j, len_l, len_h, len_I_h, len_u, len_s

(3, 4, 2, 6, 12, 10)

In [18]:
# 別々の変数が含まれるときは１つのベクトルとみなして式変形

j = 0

F = np.zeros((len_l, len_s))
for l in range(len_l):
    for s in range(len_s):
        F[l, s] = y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])

display(pd.DataFrame(F))

tmp1 = np.hstack([np.zeros((len_l, len_l)), F])
tmp2 = np.hstack([F.T, np.zeros((len_s, len_s))])

F = np.vstack([tmp1, tmp2])

display(pd.DataFrame(F))
print(np.linalg.eigvals(F))
print(f'pos. def.?: {np.all(np.linalg.eigvals(F) > 0)}')

tmp_vars = cp.hstack([lambda_j_l[j], eta_j_s[j] - eta_hat_j_s[j]])
display(tmp_vars)

# cp.quad_form(tmp_vars, F)
cp.quad_form(tmp_vars, F, assume_PSD=True)


,0,1,2,3,4,5,6,7,8,9
0,-0.26,-0.38,-0.25,-0.23,-0.19,-0.35,-0.26,-0.24,-0.43,-0.44
1,-0.24,-0.40,-0.36,-0.44,-0.44,-0.60,-0.24,-0.32,-0.44,-0.64
2,0.43,0.65,0.47,0.48,0.43,0.70,0.43,0.44,0.73,0.83
3,0.44,0.76,0.73,0.93,0.95,1.25,0.44,0.64,0.83,1.30


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00,0.00,0.00,0.00,-0.26,-0.38,-0.25,-0.23,-0.19,-0.35,-0.26,-0.24,-0.43,-0.44
1,0.00,0.00,0.00,0.00,-0.24,-0.40,-0.36,-0.44,-0.44,-0.60,-0.24,-0.32,-0.44,-0.64
2,0.00,0.00,0.00,0.00,0.43,0.65,0.47,0.48,0.43,0.70,0.43,0.44,0.73,0.83
3,0.00,0.00,0.00,0.00,0.44,0.76,0.73,0.93,0.95,1.25,0.44,0.64,0.83,1.30
4,-0.26,-0.24,0.43,0.44,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,-0.38,-0.40,0.65,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,-0.25,-0.36,0.47,0.73,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,-0.23,-0.44,0.48,0.93,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,-0.19,-0.44,0.43,0.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,-0.35,-0.60,0.70,1.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


[-3.69127742e+00+0.00000000e+00j  3.69127742e+00+0.00000000e+00j
 -4.14452631e-01+0.00000000e+00j  4.14452631e-01+0.00000000e+00j
 -1.60813882e-16+0.00000000e+00j  1.50375779e-16+0.00000000e+00j
 -8.61429478e-17+0.00000000e+00j  2.30709387e-17+0.00000000e+00j
  9.04350965e-18+1.10644825e-17j  9.04350965e-18-1.10644825e-17j
 -1.36172587e-17+6.85688105e-18j -1.36172587e-17-6.85688105e-18j
 -1.09299183e-17+0.00000000e+00j -4.90200004e-34+0.00000000e+00j]
pos. def.?: False


Expression(AFFINE, UNKNOWN, (14,))

Expression(CONVEX, NONNEGATIVE, (1, 1))

In [23]:
# cp.quad_form(arr1, F)
# cp.quad_form(arr1, F, assume_PSD=True)

####

$\sum_{l, h, i}\lambda_{j_l} \lambda_{h_i} y_l \sum_u M^h_{i, u} k_j({\bold x_l, \bold x_u})$

In [24]:
# 別々の変数が含まれるときは１つのベクトルとみなして式変形

j = 0
# h = 0

tmp_obj = 0

for h in range(len_h):
    E = np.zeros((len_l, len_I_h))
    for l in range(len_l):
        for i in range(len_I_h):
            tmp = 0
            for u in range(len_u):
                tmp += y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])
            E[l, i] = tmp

    display(pd.DataFrame(E))

    tmp1 = np.hstack([np.zeros((len_l, len_l)), E])
    tmp2 = np.hstack([E.T, np.zeros((len_I_h, len_I_h))])

    E = np.vstack([tmp1, tmp2])

    display(pd.DataFrame(E))
    print(np.linalg.eigvals(E))
    print(f'pos. def.?: {np.all(np.linalg.eigvals(E) > 0)}')

    tmp_vars = cp.hstack([lambda_j_l[j], lambda_h_i[h]])
    display(tmp_vars)

    # cp.quad_form(tmp_vars, E)
    tmp_obj += cp.quad_form(tmp_vars, E, assume_PSD=True)

print("hello")
display(tmp_obj)

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pos. def.?: False


Expression(AFFINE, NONNEGATIVE, (10,))

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pos. def.?: False


Expression(AFFINE, NONNEGATIVE, (10,))

hello


Expression(CONVEX, NONNEGATIVE, (1, 1))

In [26]:
print(tmp_obj)

QuadForm(Hstack(var1[0, 0:4], var2[0, 0:6]), psd_wrap([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]])) + QuadForm(Hstack(var1[0, 0:4], var2[1, 0:6]), psd_wrap([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]))


In [29]:
len_l, len_u, len_s

(4, 12, 10)

####

$\sum_{h, i, s} \lambda_{h_i} (\eta_{j_s} - \hat \eta_{j_s}) \sum_u M^h_{i, u} k_j(\bold x_u, \bold x_s)$

In [28]:
# 別々の変数が含まれるときは１つのベクトルとみなして式変形

j = 0
# h = 0

tmp_obj = 0

for h in range(len_h):
    G = np.zeros((len_I_h, len_s))
    for i in range(len_I_h):
        for s in range(len_s):
            tmp = 0
            for u in range(len_u):
                tmp += M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
            G[i, s] = tmp

    display(pd.DataFrame(G))

    tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), G])
    tmp2 = np.hstack([G.T, np.zeros((len_s, len_s))])

    G = np.vstack([tmp1, tmp2])

    display(pd.DataFrame(G))
    print(np.linalg.eigvals(G))
    print(f'pos. def.?: {np.all(np.linalg.eigvals(G) > 0)}')

    tmp_vars = cp.hstack([lambda_h_i[h], eta_j_s[j] - eta_hat_j_s[j]])
    display(tmp_vars)

    # cp.quad_form(tmp_vars, G)
    tmp_obj += cp.quad_form(tmp_vars, G, assume_PSD=True)

print("hello")
display(tmp_obj)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pos. def.?: False


Expression(AFFINE, UNKNOWN, (16,))

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pos. def.?: False


Expression(AFFINE, UNKNOWN, (16,))

hello


Expression(CONVEX, NONNEGATIVE, (1, 1))

In [22]:
x_S

[array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.4, 0.3, 0.9],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.4, 0.8, 0.7]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.6, 0.2, 0.7],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.3, 0.4, 0.8, 0.6]]),
 array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.4, 0.9, 0.2, 0.5],
        [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.2, 0.3, 0.6, 0.7]])]

####

$\sum_{h, h', i, i'} \lambda_{h_i}, \lambda_{{h'}_{i'}} \sum_{u, u'} M^h_{i, u} M^{h'}_{i', u'} k_j(\bold x_u, \bold x_{u'})$

In [241]:
x_U_tmp

array([[0.1, 0.3, 0.5, 0.8, 0.9, 1. , 0.1, 0.3, 0.5, 0.8, 0.9, 1. ],
       [0.5, 0.7, 0.4, 0.3, 0.2, 0.5, 0.5, 0.7, 0.4, 0.3, 0.2, 0.5]])

In [273]:
# 別々の変数が含まれるときは１つのベクトルとみなして式変形

j = 0
# h = 0

tmp_obj = 0

for h in range(len_h):
    for h_prime in range(len_h):
        # 各 h について，I_h （h が取る要素の数）がそれぞれ違えば，ここは変更しないといけない
        B = np.zeros((len_I_h, len_I_h))
        for i in range(len_I_h):
            for i_prime in range(len_I_h):
                for u in range(len_u):
                    for u_prime in range(len_u):
                        B[i, i_prime] = M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])

        display(pd.DataFrame(B))

        tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), B])
        tmp2 = np.hstack([B.T, np.zeros((len_I_h, len_I_h))])

        B = np.vstack([tmp1, tmp2])

        display(pd.DataFrame(B))
        print(np.linalg.eigvals(B))
        print(f'pos. def.?: {np.all(np.linalg.eigvals(B) > 0)}')

        tmp_vars = cp.hstack([lambda_h_i[h], lambda_h_i[h_prime]])
        display(tmp_vars)

        cp.quad_form(tmp_vars, B)
        # tmp_obj += cp.quad_form(tmp_vars, B)
        tmp_obj += cp.quad_form(tmp_vars, B, assume_PSD=True)

print("hello")
display(tmp_obj)

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.0,0.0,1.25


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.25
6,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
8,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
9,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00


[ 1.25 -1.25  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
pos. def.?: False


Expression(AFFINE, NONNEGATIVE, (12,))

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.0,0.0,1.25


...

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.25
6,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
8,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
9,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00


[ 1.25 -1.25  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
pos. def.?: False


Expression(AFFINE, NONNEGATIVE, (12,))

hello


Expression(CONVEX, NONNEGATIVE, (1, 1))

### 1.2.2 目的関数 2

* 二次形式の挟まれる行列の構成方法を工夫

In [29]:
# 通常の内積

def kernel_function(x1, x2):
    return x1.T @ x2

k = kernel_function


# 変数 
lambda_j_l = cp.Variable(shape=(3, 4), nonneg=True) # j \in {1, 2, 3}, l \in {1, 2, 3, 4}
lambda_h_i = cp.Variable(shape=(2, 6), nonneg=True) # h \in {1, 2}, i \in {1, 2, 3, 4, 5, 6}
eta_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
eta_hat_j_s = cp.Variable(shape=(3, 10), nonneg=True) # j \in {1, 2, 3}, s \in {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

len_j = 3
len_l = 4 

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])

In [30]:
obj_func = 0


for j in range(len_j):
    A = np.zeros((len_l, len_l))
    for l in range(len_l):
        for l_prime in range(len_l):
            A[l, l_prime] = y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])
    # print(f'A \n{A}\n')

    obj_func += 4 * cp.quad_form(lambda_j_l[0, :], A)


    tmp_obj = 0
    for h in range(len_h):
        for h_prime in range(len_h):
            # 各 h について，I_h （h が取る要素の数）がそれぞれ違えば，ここは変更しないといけない
            B = np.zeros((len_I_h, len_I_h))
            for i in range(len_I_h):
                for i_prime in range(len_I_h):
                    for u in range(len_u):
                        for u_prime in range(len_u):
                            B[i, i_prime] = M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])
            tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), B])
            tmp2 = np.hstack([B.T, np.zeros((len_I_h, len_I_h))])
            B = np.vstack([tmp1, tmp2])
            tmp_vars = cp.hstack([lambda_h_i[h], lambda_h_i[h_prime]])
            cp.quad_form(tmp_vars, B)
            # tmp_obj += cp.quad_form(tmp_vars, B)
            tmp_obj += cp.quad_form(tmp_vars, B, assume_PSD=True)
    
    obj_func += tmp_obj


    D = np.zeros((len_s, len_s))
    for s in range(len_s):
        for s_prime in range(len_s):
            D[s, s_prime] = k(x_S[j][:, s], x_S[j][:, s_prime])
    # print(f'D \n{D}\n')
    
    obj_func += cp.quad_form(eta_j_s[j, :] - eta_hat_j_s[j, :], D)


    tmp_obj = 0
    for h in range(len_h):
        E = np.zeros((len_l, len_I_h))
        for l in range(len_l):
            for i in range(len_I_h):
                tmp = 0
                for u in range(len_u):
                    tmp += y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])
                E[l, i] = tmp
        tmp1 = np.hstack([np.zeros((len_l, len_l)), E])
        tmp2 = np.hstack([E.T, np.zeros((len_I_h, len_I_h))])
        E = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_j_l[j], lambda_h_i[h]])
        # cp.quad_form(tmp_vars, E)
        tmp_obj += cp.quad_form(tmp_vars, E, assume_PSD=True)
    
    # ここで -4 でかけると 全体で concavity がなくなる
    # obj_func += (-4) * tmp_obj
    obj_func += 4 * tmp_obj


    F = np.zeros((len_l, len_s))
    for l in range(len_l):
        for s in range(len_s):
            F[l, s] = y_L[j][0, l] * k(x_L[j][:, l], x_S[j][:, s])
    tmp1 = np.hstack([np.zeros((len_l, len_l)), F])
    tmp2 = np.hstack([F.T, np.zeros((len_s, len_s))])
    F = np.vstack([tmp1, tmp2])
    tmp_vars = cp.hstack([lambda_j_l[j], eta_j_s[j] - eta_hat_j_s[j]])

    # obj_func += 4 * cp.quad_form(tmp_vars, F)
    obj_func += 4 * cp.quad_form(tmp_vars, F, assume_PSD=True)
    

    tmp_obj = 0
    for h in range(len_h):
        G = np.zeros((len_I_h, len_s))
        for i in range(len_I_h):
            for s in range(len_s):
                tmp = 0
                for u in range(len_u):
                    tmp += M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])
                G[i, s] = tmp
        tmp1 = np.hstack([np.zeros((len_I_h, len_I_h)), G])
        tmp2 = np.hstack([G.T, np.zeros((len_s, len_s))])
        G = np.vstack([tmp1, tmp2])
        tmp_vars = cp.hstack([lambda_h_i[h], eta_j_s[j] - eta_hat_j_s[j]])
        # cp.quad_form(tmp_vars, G)
        tmp_obj += cp.quad_form(tmp_vars, G, assume_PSD=True)

    # ここで，-2 でかけると，全体で concavity がなくなる
    # obj_func += (-2) * tmp_obj
    obj_func += 2 * tmp_obj


for j in range(len_j):
    for l in range(len_l):
        obj_func += lambda_j_l[j, l]

for h in range(len_h):
    for i in range(len_I_h):
        tmp = 0
        for u in range(len_u):
            tmp += M[h][i, u]
        # 
        obj_func += lambda_h_i[h, i] * (1/2 * tmp + q[h][i,0])

tmp = 0
for j in range(len_j):
    for s in range(len_s):
        tmp += eta_j_s[j, s] + eta_hat_j_s[j, s]
obj_func += -1/2 * tmp


obj_func = (-1/2) * obj_func

display(obj_func)

Expression(CONCAVE, UNKNOWN, (1, 1))

## 1.2 制約



In [31]:
constraint_1 = 0

for j in range(len_j):
    for h in range(len_h):
        for i in range(len_I_h):
            for u in range(len_u):
                constraint_1 += lambda_h_i[h, i] * M[h][i, u]

    for l in range(len_l):
        constraint_1 += (-2) * lambda_j_l[j, l] * y_L[j][0, l]

    for s in range(len_s):
        constraint_1 += eta_j_s[j, s] - eta_hat_j_s[j, s]

display(constraint_1)

Expression(AFFINE, UNKNOWN, ())

## 1.3 最適化

In [33]:
C_1, C_2 = 2.5, 2.5

objective = cp.Maximize(obj_func)

constraints = [
    constraint_1 == 0,
    lambda_j_l >=0,
    lambda_j_l <= C_1,
    lambda_h_i >= 0,
    lambda_h_i <= C_2,
    eta_j_s >=0,
    eta_hat_j_s >= 0
]


problem = cp.Problem(objective, constraints)
# result = problem.solve()

In [34]:
result = problem.solve()

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


SolverError: Workspace allocation error!

In [17]:
result = problem.solve('SOC')

SolverError: The solver SOC is not installed.

In [18]:
result = problem.solve('SCIPY')


SolverError: Either candidate conic solvers (['SCIPY']) do not support the cones output by the problem (SOC, NonNeg, Zero), or there are not enough constraints in the problem.

In [19]:
result = problem.solve('CVXOPT')

/home/onoue/ws/lukasiewicz/myenv/lib/python3.10/site-packages/cvxpy/atoms/quad_form.py:219: UserWarning: Forming a nonconvex expression quad_form(x, indefinite).
  warnings.warn("Forming a nonconvex expression quad_form(x, indefinite).")


In [32]:
cp.settings.SOLVERS

['CLARABEL',
 'ECOS',
 'CVXOPT',
 'GLOP',
 'GLPK',
 'GLPK_MI',
 'SCS',
 'SDPA',
 'GUROBI',
 'OSQP',
 'CPLEX',
 'MOSEK',
 'CBC',
 'COPT',
 'XPRESS',
 'PROXQP',
 'NAG',
 'PDLP',
 'SCIP',
 'SCIPY']

## 旧

In [14]:
len_j = 3
len_l = 4 

len_h = 2
len_I_h = 6

len_u = 12

len_s = 10

# とりあえずこれで
x_U_tmp = np.hstack([x_U[:2][1], x_U[1:][0]])
x_U_tmp


obj_func = 0

for j in range(len_j):
    for l in range(len_l):
        for l_prime in range(len_l):
            obj_func += 4 * lambda_j_l[j, l] * lambda_j_l[j, l_prime] * y_L[j][0, l] * y_L[j][0, l_prime] * k(x_L[j][:, l], x_L[j][:, l_prime])
    

    # M が 0 のところを条件分岐してみる
    for h in range(len_h):
        for h_prime in range(len_h):
            for i in range(len_I_h):
                for i_prime in range(len_I_h):
                    for u in range(len_u):
                        for u_prime in range(len_u):
                            if M[h][i, u] == 0 or M[h_prime][i_prime, u_prime]:
                                pass
                            else:
                                obj_func += lambda_h_i[h, i] * lambda_h_i[h_prime, i_prime] * M[h][i, u] * M[h_prime][i_prime, u_prime] * k(x_U_tmp[:, u], x_U_tmp[:, u_prime])

    for s in range(len_s):
        for s_prime in range(len_s):
            obj_func += (eta_j_s[j, s] - eta_hat_j_s[j, s]) * (eta_j_s[j, s_prime] - eta_hat_j_s[j, s_prime]) * k(x_S[j][:, s], x_S[j][:, s_prime])

    for l in range(len_l):
        for h in range(len_h):
            for i in range(len_I_h):
                for u in range(len_u):
                        # if M[h][i, u] == 0 or M[h_prime][i_prime, u_prime]:
                        #     pass
                        # else:
                        obj_func += (-4) * lambda_j_l[j, l] * lambda_h_i[h, i] * y_L[j][0, l] * M[h][i, u] * k(x_L[j][:, l], x_U_tmp[:, u])

    for l in range(len_l):
        for s in range(len_s):
            obj_func += 4 * lambda_j_l[j, l] * y_L[j][0, l] * (eta_j_s[j, s] - eta_hat_j_s[j, s]) * k(x_L[j][:, l], x_S[j][:, s])

    for h in range(len_h):
        for i in range(len_I_h):
            for s in range(len_s):
                for u in range(len_u):
                    obj_func += (-2) * lambda_h_i[h, i] * (eta_j_s[j, s] - eta_hat_j_s[j, s]) * M[h][i, u] * k(x_U_tmp[:, u], x_S[j][:, s])

print(obj_func)

obj_func = (-1/2) * obj_func

for j in range(len_j):
    for l in range(len_l):
        obj_func += lambda_j_l[j, l]

for h in range(len_h):
    for i in range(len_I_h):
        for u in range(len_u):
            obj_func += lambda_h_i[h, i] * (1/2 * M[h][i, u] + q[h][i, 0])

for j in range(len_j):
    for s in range(len_s):
        obj_func += (-1/2) * (eta_j_s[j, s] - eta_hat_j_s[j, s])

# print(obj_func)

4.0 @ var1[0, 0] @ var1[0, 0] @ -1.0 @ -1.0 @ 0.26 + 4.0 @ var1[0, 0] @ var1[0, 1] @ -1.0 @ -1.0 @ 0.24000000000000002 + 4.0 @ var1[0, 0] @ var1[0, 2] @ -1.0 @ 1.0 @ 0.43000000000000005 + 4.0 @ var1[0, 0] @ var1[0, 3] @ -1.0 @ 1.0 @ 0.44 + 4.0 @ var1[0, 1] @ var1[0, 0] @ -1.0 @ -1.0 @ 0.24000000000000002 + 4.0 @ var1[0, 1] @ var1[0, 1] @ -1.0 @ -1.0 @ 0.32000000000000006 + 4.0 @ var1[0, 1] @ var1[0, 2] @ -1.0 @ 1.0 @ 0.44000000000000006 + 4.0 @ var1[0, 1] @ var1[0, 3] @ -1.0 @ 1.0 @ 0.64 + 4.0 @ var1[0, 2] @ var1[0, 0] @ 1.0 @ -1.0 @ 0.43000000000000005 + 4.0 @ var1[0, 2] @ var1[0, 1] @ 1.0 @ -1.0 @ 0.44000000000000006 + 4.0 @ var1[0, 2] @ var1[0, 2] @ 1.0 @ 1.0 @ 0.7300000000000001 + 4.0 @ var1[0, 2] @ var1[0, 3] @ 1.0 @ 1.0 @ 0.83 + 4.0 @ var1[0, 3] @ var1[0, 0] @ 1.0 @ -1.0 @ 0.44 + 4.0 @ var1[0, 3] @ var1[0, 1] @ 1.0 @ -1.0 @ 0.64 + 4.0 @ var1[0, 3] @ var1[0, 2] @ 1.0 @ 1.0 @ 0.83 + 4.0 @ var1[0, 3] @ var1[0, 3] @ 1.0 @ 1.0 @ 1.3 + var2[0, 0] @ var2[0, 0] @ -1.0 @ 0.0 @ 0.38 + var2

In [54]:
constraint_1 = 0

for j in range(len_j):
    for h in range(len_h):
        for i in range(len_I_h):
            for u in range(len_u):
                constraint_1 += lambda_h_i[h, i] * M[h][i, u]

    for l in range(len_l):
        constraint_1 += (-2) * lambda_j_l[j, l] * y_L[j][0, l]

    for s in range(len_s):
        constraint_1 += eta_j_s[j, s] - eta_hat_j_s[j, s]

In [55]:
C_1, C_2 = 2.5, 2.5

objective = cp.Maximize(obj_func)

constraints = [
    constraint_1 == 0,
    lambda_j_l >=0,
    lambda_j_l <= C_1,
    lambda_h_i >= 0,
    lambda_h_i <= C_2,
    eta_j_s >=0,
    eta_hat_j_s >= 0
]


problem = cp.Problem(objective, constraints)
result = problem.solve()

/home/onoue/ws/lukasiewicz/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
4.0 @ var1[0, 0] @ var1[0, 0]
4.0 @ var1[0, 0] @ var1[0, 1]
4.0 @ var1[0, 0] @ var1[0, 2]
4.0 @ var1[0, 0] @ var1[0, 3]
4.0 @ var1[0, 1] @ var1[0, 0]
4.0 @ var1[0, 1] @ var1[0, 1]
4.0 @ var1[0, 1] @ var1[0, 2]
4.0 @ var1[0, 1] @ var1[0, 3]
4.0 @ var1[0, 2] @ var1[0, 0]
4.0 @ var1[0, 2] @ var1[0, 1]
4.0 @ var1[0, 2] @ var1[0, 2]
4.0 @ var1[0, 2] @ var1[0, 3]
4.0 @ var1[0, 3] @ var1[0, 0]
4.0 @ var1[0, 3] @ var1[0, 1]
4.0 @ var1[0, 3] @ var1[0, 2]
4.0 @ var1[0, 3] @ var1[0, 3]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ var2[0, 0]
var2[0, 0] @ 

In [ ]:
# # 必要ライブラリの読み込み
# import pandas as pd
# from scipy.io import loadmat
# matdata = loadmat(u'変換したいmatファイル.mat')

# # 一時変換用の辞書
# dictFromMat = {}
# # 辞書の要素を列挙し、DataFrameに変換
# for key, value in matdata.iteritems():
#     dictFromMat[key] = pd.Series(np.array(matdata[key]).flatten())
   
# # 一時変換用の辞書からDataFrameを作成
# dfFromMat = pd.DataFrame(dictFromMat)

In [142]:
for i in range(lambda_h_i.shape[0]):
    for j in range(lambda_h_i.shape[1]):
        print(lambda_h_i[i][j])

    print()

var212[0, 0:6][0]
var212[0, 0:6][1]
var212[0, 0:6][2]
var212[0, 0:6][3]
var212[0, 0:6][4]
var212[0, 0:6][5]

var212[1, 0:6][0]
var212[1, 0:6][1]
var212[1, 0:6][2]
var212[1, 0:6][3]
var212[1, 0:6][4]
var212[1, 0:6][5]



In [19]:
3 * 4 * 36 * 144

62208

In [122]:
print(x)


[[0.1 0.4 0.3 0.9]
 [0.5 0.4 0.8 0.7]
 [0.1 0.6 0.2 0.7]
 [0.3 0.4 0.8 0.6]
 [0.4 0.9 0.2 0.5]
 [0.2 0.3 0.6 0.7]]


In [ ]:
for i in range(3)


In [131]:
lambda_h_i[0].shape

(6,)

In [132]:
lambda_h_i.shape

(2, 6)

In [136]:
k(np.array([0,2]), np.array([3,4]))

8

In [77]:
for i in lambda_h_i:
    print(i)

var50[0, 0:6]
var50[1, 0:6]


In [78]:
lambda_h_i[0][0]

Expression(AFFINE, UNKNOWN, ())

In [82]:
for i in range(lambda_h_i.shape[0]):
    for j in range(lambda_h_i.shape[1]):
        print(lambda_h_i[i][j] * 2)

var50[0, 0:6][0] @ 2.0
var50[0, 0:6][1] @ 2.0
var50[0, 0:6][2] @ 2.0
var50[0, 0:6][3] @ 2.0
var50[0, 0:6][4] @ 2.0
var50[0, 0:6][5] @ 2.0
var50[1, 0:6][0] @ 2.0
var50[1, 0:6][1] @ 2.0
var50[1, 0:6][2] @ 2.0
var50[1, 0:6][3] @ 2.0
var50[1, 0:6][4] @ 2.0
var50[1, 0:6][5] @ 2.0


In [84]:
obj_func = 0

for i in range(lambda_h_i.shape[0]):
    for j in range(lambda_h_i.shape[1]):
        obj_func += lambda_h_i[i][j] * 2

print(obj_func)

var50[0, 0:6][0] @ 2.0 + var50[0, 0:6][1] @ 2.0 + var50[0, 0:6][2] @ 2.0 + var50[0, 0:6][3] @ 2.0 + var50[0, 0:6][4] @ 2.0 + var50[0, 0:6][5] @ 2.0 + var50[1, 0:6][0] @ 2.0 + var50[1, 0:6][1] @ 2.0 + var50[1, 0:6][2] @ 2.0 + var50[1, 0:6][3] @ 2.0 + var50[1, 0:6][4] @ 2.0 + var50[1, 0:6][5] @ 2.0


In [75]:
# 変数
x_1 = cp.Variable()
x_2 = cp.Variable()

objective = cp.Minimize(
    cp.power(x_1, 2) + 2 * cp.power(x_2, 2)
)

constraints = [
    x_2 >= - x_1 + 1,
    x_2 >= (x_1 - 1) / 2.0
]

problem = cp.Problem(objective, constraints)
result = problem.solve()
print(result) 
print(x_1.value, x_2.value)
print(constraints[0].dual_value, constraints[1].dual_value)  

0.6666666666666667
0.6666666666666667 0.33333333333333337
1.3333333333333337 0.0


# 2. Winston benchmark